## Introduction

In this activity we will plot a few common relative permeability functional forms and explore a common approach for calculating relative permeability from capillary pressure curves.

Start by importing our necessary packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
import ipywidgets as widgets

## Brooks-Corey relative permeability function

In the [Capillary Pressure Functions notebook](https://github.com/zahasky/Contaminant-Hydrogeology-Activities/blob/master/Capillary%20Pressure%20Functions.ipynb) we explored a few common capillay pressure functional forms. Both the van Genuchten and Brooks-Corey capillary pressure models have corresponding relative permeability functional forms. The Brooks-Corey empirical equations are programmed in the following function:

In [ ]:
# Brooks-Corey function
def brooks_corey_rel_perm_fun(Sw, Swr, Snwr, krw0, krnw0, m):
    # Now calculate the effective saturation (think of this as normalized saturation (ranges from 0-1))
    Se = (Sw - Swr)/((1 - Snwr) - Swr)
    # Water relative permeability
    krw = krw0*Se**(2/m + 3)
    # Nonwetting phase relative permeability
    krnw = krnw0*(1-Se)**2*(1-Se**(2/m + 1))
    return krw, krnw

When deciding which model to use it is best to start with the most simple model and move to more complex functions as neccessary to fit your measurement data. Often a modified (simplified) Brooks-Corey functional form is sufficient.

In [ ]:
# Modified Brooks-Corey function
def mod_brooks_corey_rel_perm_fun(Sw, Swr, Snwr, krw0, krnw0, nw, nnw):
    # Now calculate the effective saturation (think of this as normalized saturation (ranges from 0-1))
    Se = (Sw - Swr)/((1 - Snwr) - Swr)
    # Water relative permeability
    krw = krw0*Se**nw
    # Nonwetting phase relative permeability
    krnw = krnw0*(1-Se)**nnw
    return krw, krnw

As you can see, there are strong similiarities with the classic Brooks-Corey model but because of the simplicity of the exponential form, this model is also more intuative. 

In [ ]:
# Define residual water saturation (the residual water after drainage)
Swr = 0.1
# Define the residual gas saturation (defined as the water saturation after imbibition)
Snwr = 0
# define variable 'Sw' to describe water saturation
Sw = np.linspace((Swr + 0.001), (1 - Snwr - 0.001), num=100)

# End point relative permeability. 
# Water end point relative permeability, this is equal to the relative permeabiltiy at the residual gas saturation
krw0 = 1
# Nonwetting phase end point relative permeability
krnw0 = 1

# Call Brooks-Corey function
krw_bc, krnw_bc = brooks_corey_rel_perm_fun(Sw, Swr, Snwr, krw0, krnw0, 1)
# Call modified Brooks-Corey function
krw_mbc, krnw_mbc = mod_brooks_corey_rel_perm_fun(Sw, Swr, Snwr, krw0, krnw0, 3, 3)

# Plot the results
plt.plot(Sw, krw_bc, label='Brooks-Corey Water Rel Perm')
plt.plot(Sw, krnw_bc, label='Brooks-Corey Nonwetting Rel Perm')

plt.plot(Sw, krw_mbc, label='Mod BC Water Rel Perm')
plt.plot(Sw, krnw_mbc, label='Mod BC Nonwetting Rel Perm')

plt.xlabel('Water Saturation')
plt.ylabel('Relative Permeability')
plt.legend()
plt.xlim(0, 1)
plt.show()

##### Increase the exponent in the modified Brooks-Corey curves. How do the relative permeability curves change? How does this change in the relative permeability translate to the physical system? 

Note that sometimes when the exponents are very high and the relative permeability values are very small, you may see relative permability plotted on log scale on the y-axis. You could explore the differences by adding the following code to your plot script:

    plt.yscale('log')
    plt.ylim(0.001, 1) # adjust for data range

## Burdine's Method to calculate relative permeability from a capillary pressure curve
It is usually much easier to measure capillary pressure than relative permeability so it is convenient to have a method to calculate relative permeability from a capillary pressure curve. Burdine's method is most commonly used to accomplish this. In this activity you will see how Burdine's method enables us to use a capillary pressure curve to calculate the corresponding relative permeability curves. You can see the theoretical derivation of this Brooks-Corey model in the coures notes.

Let's start with the Brooks-Corey capillary pressure function that we saw in the [Capillary Pressure Functions notebook](https://github.com/zahasky/Contaminant-Hydrogeology-Activities/blob/master/Capillary%20Pressure%20Functions.ipynb).

In [ ]:
# Brooks-Corey capillary pressure function 
def brooks_corey_pc(Sw, Swr, Snwr, Pc_entry, m):
    # Now calculate the effective saturation (think of this as normalized saturation (ranges from 0-1))
    Se = (Sw - Swr)/((1 - Snwr) - Swr)
    Pc = Pc_entry*(Se**(-1/m))
    return Pc

Now let's plot an example capillary pressure curve.

In [ ]:
# Define residual water saturation (the residual water after drainage)
Swr = 0.1
# Define the residual gas saturation (defined as the water saturation after imbibition)
Snwr = 0
# define variable 'Sw' to describe water saturation
Sw = np.linspace((Swr + 0.001), (1 - Snwr - 0.001), num=100)

m_bc = 3
Pc_entry_bc = 0.9
Pc_bc = brooks_corey_pc(Sw, Swr, Snwr, Pc_entry_bc, m_bc)

plt.plot(Sw, Pc_bc)
plt.xlabel('Water Saturation')
plt.ylabel('Capillary Pressure (kPa)')
plt.show()

### Illustration of Burdine's method
This cell illustrates Burdine's method for a given value of Sw (as choosen by sliding the water saturation index). The full equations for this are given in the relative permeability section of Chapter 3 of the course notes.

In [ ]:
# Denominator (constant)
denom = np.trapezoid(1 / Pc_bc**2, Sw)

# Interactive function
def update(i):
    Sw_val = Sw[i]
    Se_val = (Sw_val - Swr)/((1 - Snwr) - Swr)
    
    kw_numer = 1 / Pc_bc[:i]**2
    knw_numer = 1 / Pc_bc[i:]**2

    krw_integrand = Se_val**2 * np.trapezoid(kw_numer, Sw[:i])
    krw_val = krw_integrand / denom

    knrw_integrand = (1 - Se_val)**2 * np.trapezoid(knw_numer, Sw[i:])
    knrw_val = knrw_integrand / denom
    
    fig, axs = plt.subplots(1, 3, figsize=(9, 3.5))
    fig.suptitle(f"Burdine Relative Permeability at $S_w = {Sw_val:.2f}$", fontsize=14)
    
    # Plot 1: Denominator
    axs[0].plot(Sw, 1 / Pc_bc**2, label=f'area = {denom:.2f}', color='blue')
    axs[0].fill_between(Sw, 1 / Pc_bc**2, color='blue', alpha=0.3)
    axs[0].set_title("Denominator")
    axs[0].set_xlabel("Sw")
    axs[0].set_ylabel(r'$1/P_c^2$')
    axs[0].set_xlim([0, 1])
    axs[0].legend()
    
    # Plot 2: krw integrand
    axs[1].plot(Sw[:i], kw_numer, label=f'area = {krw_integrand:.3f}', color='green')
    axs[1].fill_between(Sw[:i], kw_numer, color='green', alpha=0.3)
    axs[1].set_title(f"Integrand for $k_{{rw}}$\n$k_{{rw}} = {krw_val:.3f}$")
    axs[1].set_xlabel("Sw")
    axs[1].set_ylabel(r'$1/P_c^2$')
    axs[1].set_xlim([0, 1])
    axs[1].legend()
    
    # Plot 3: knrw integrand
    axs[2].plot(Sw[i:], knw_numer, label=f'area = {knrw_integrand:.3f}', color='red')
    axs[2].fill_between(Sw[i:], knw_numer, color='red', alpha=0.3)
    axs[2].set_title(f"Integrand for $k_{{nrw}}$\n$k_{{nrw}} = {knrw_val:.3f}$")
    axs[2].set_xlabel("Sw")
    axs[2].set_ylabel(r'$1/P_c^2$')
    axs[2].set_xlim([0, 1])
    axs[2].legend()
    
    plt.tight_layout()
    plt.show()

# Create slider
slider = IntSlider(min=5, max=len(Sw) - 5, step=1, value=50, description='Sw index')

# Display interactive widget
interact(update, i=slider)


Use this slider functionality to sketch out (on paper) what the relative permeability curve looks like, use at least 5 values of water saturation. If the widget functionality is not working in your python environment, then you can also run the cell below and rerun the cell with different values of `i`.

In [ ]:
# Choose an index to visualize
i = 50
Sw_val = Sw[i]
# Normalized saturation
Se_val = (Sw_val - Swr)/((1 - Snwr) - Swr)

# Denominator is the same for both
denom = np.trapezoid(1 / Pc_bc**2, Sw)

# Compute numerators for krw and krnw
kw_numer = 1 / Pc_bc[:i]**2
knw_numer = 1 / Pc_bc[i:]**2

krw_integrand = Se_val**2 * np.trapezoid(kw_numer, Sw[:i])
krw_val = krw_integrand / denom

knrw_integrand = (1 - Se_val)**2 * np.trapezoid(knw_numer, Sw[i:])
knrw_val = knrw_integrand / denom

# Plotting
fig, axs = plt.subplots(1, 3, figsize=(9, 3))
fig.suptitle(f"Burdine Relative Permeability Calculation at Sw = {Sw_val:.2f}", fontsize=14)

# 1. krw Integrand
axs[0].plot(Sw, 1 / Pc_bc**2, label=f'area = {denom:.2f}', color='blue')
axs[0].fill_between(Sw, 1 / Pc_bc**2, color='blue', alpha=0.3)
axs[0].set_title("Denominator")
axs[0].set_xlabel("Sw")
axs[0].set_ylabel(r'$1/P_c^2$')
axs[0].set_xlim([0,1])
axs[0].legend()

# 2. krw Integrand
axs[1].plot(Sw[:i], kw_numer, label=f'area = {krw_integrand:.3f}', color='green')
axs[1].fill_between(Sw[:i], kw_numer, color='green', alpha=0.3)
axs[1].set_title(f"Integrand for $k_{{rw}}$, $k_{{rw}} = {krw_val:.3f}$")
axs[1].set_xlabel("Sw")
axs[1].set_ylabel(r'$1/P_c^2$')
axs[1].set_xlim([0,1])
axs[1].legend()

# 3. krnw Integrand
axs[2].plot(Sw[i:], knw_numer, label=f'area = {knrw_integrand:.3f}', color='red')
axs[2].fill_between(Sw[i:], knw_numer, color='red', alpha=0.3)
axs[2].set_title(f"Integrand for $k_{{nrw}}$, $k_{{nrw}} = {knrw_val:.3f}$")
axs[2].set_xlabel("Sw")
axs[2].set_ylabel(r'$1/P_c^2$')
axs[2].set_xlim([0,1])
axs[2].legend()

plt.tight_layout()
plt.show()

Now define a function that takes in water saturation and capillary pressure and calculates wetting and nonwetting relative permeability using Burdine's method as illustrated in the previous cell.

In [ ]:
# Burdine function
def burdine_fun(Sw, Swr, Snwr, Pc):
    # Normalized saturation
    Se = (Sw - Swr)/((1 - Snwr) - Swr)
    # both of the relative permeability integrals have the same fixed denominator
    denom = np.trapezoid(1/Pc**2, Sw)
    # preallocate the array for saving the values
    krw_burdine = np.zeros(np.shape(Sw))
    krnw_burdine = np.zeros(np.shape(Sw))
    
    # integrate from Swr to Sw
    for i in range(len(Sw)-1,0,-1):
        kw_numer = 1/Pc[:i]**2
        krw_burdine[i] = Se[i]**2*np.trapezoid(kw_numer, Sw[:i])/denom
    
    # integrate from Sw to 1    
    for i in range(len(Sw)):
        knw_numer = 1/Pc[i:]**2
        krnw_burdine[i] = (1-Se[i])**2*np.trapezoid(knw_numer, Sw[i:])/denom
    
        ## Add plot showing areas for visualization
    return krw_burdine, krnw_burdine

Now let's see if this works by calculating the Brooks-Corey relative permeability using the same ```m``` as we used for defining capillary pressure (```Pc_bc```).

In [ ]:
# Calculate brooks-corey rel perm
krw_bc, krnw_bc = brooks_corey_rel_perm_fun(Sw, Swr, Snwr, krw0, krnw0, m_bc)

# Calculate rel perm from brooks-corey capillary pressure curve
krw_burdine, krnw_burdine = burdine_fun(Sw, Swr, Snwr, Pc_bc)

plt.plot(Sw, krw_bc, label='Brooks-Corey Water Rel Perm')
plt.plot(Sw, krnw_bc, label='Brooks-Corey Nonwetting Rel Perm')

plt.plot(Sw, krw_burdine, '--', label='Burdine Water Rel Perm')
plt.plot(Sw, krnw_burdine, '--', label='Burdine Nonwetting Rel Perm')

plt.xlabel('Water Saturation')
plt.ylabel('Relative Permeability')
plt.legend()
plt.xlim(0, 1)
plt.show()

Pretty cool, right?!

## Activity: Use Burdine's method to calculate relative permeability from capillary pressure data

##### Now, using the capillary pressure function that you fit to the capillary pressure data in the [Capillary Pressure Curves notebook](https://github.com/zahasky/Contaminant-Hydrogeology-Activities/blob/master/Capillary%20Pressure%20Curves.ipynb), calculate the corresponding relative permeability curves using Burdine's Method.